In [13]:
import pandas as pd
import numpy as np
import random
from quantnote import BacktestEngine, Strategy

In [2]:
db_path = '/Users/j.nahmgoong/Documents/git/DB/MainDB.db'

engine = BacktestEngine(db_path)

Loading DB...
DB loaded in 44.48 seconds
[initialize] is executed in 50.66 seconds


In [3]:
engine.cache['fundamentals']['AAPL'][['ticker','assets','gp','rnd']]

,ticker,assets,gp,rnd
datekey,,,,
1997-12-05,AAPL,4.233000e+09,1.368000e+09,4.850000e+08
1998-02-09,AAPL,4.126000e+09,NaN,NaN
1998-05-11,AAPL,3.963000e+09,NaN,NaN
1998-08-10,AAPL,4.041000e+09,NaN,NaN
1998-12-23,AAPL,4.289000e+09,1.479000e+09,3.030000e+08
...,...,...,...,...
2020-07-31,AAPL,3.173440e+11,1.045800e+11,1.788400e+10
2020-10-30,AAPL,3.238880e+11,1.049560e+11,1.875200e+10
2021-01-28,AAPL,3.540540e+11,1.140670e+11,1.946400e+10


In [4]:
engine.show_sample_strategy()

class myStrategy(Strategy):
    def __init__(self):
        super().__init__()

    def compute_target(self, universe_list):
        target_weight = { }
        for ticker in universe_list:
            target_weight[ticker] = 1
        target_weight = self.normalize(target_weight)
        return target_weight

    def custom_factor(self, ticker, ftype):
        if ftype == 'marketcap':
            return self.get_value('metric',ticker,'marketcap')
        else:
            assert False



In [27]:
class myStrategy(Strategy):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        print(self.config)

    def compute_target(self, universe_list):
        target_weight = { }
        
        input1 = self.config['input1']
        input2 = self.config['input2']
        unary = self.config['unary']
        binary = self.config['binary']
        
        factor_series_1 = self.compute_factor_series(universe_list, input1, type=unary)
        factor_series_2 = self.compute_factor_series(universe_list, input2, type=unary)
        
        if binary == 'add':
            factor_series = factor_series_1 + factor_series_2
        elif binary == 'sub':
            factor_series = factor_series_1 - factor_series_2
        elif binary == 'mul':
            factor_series = factor_series_1 * factor_series_2
        elif binary == 'div':
            factor_series = factor_series_1 / factor_series_2
            
        
        factor_series = factor_series.dropna().sort_values(ascending=False)
        
        assert len(factor_series) > 0.5*len(universe_list)
        
        top = int(len(factor_series.index)/5)
        
        for ticker in factor_series.index[:top]:
            target_weight[ticker] = 1
            
        target_weight = self.normalize(target_weight)
        return target_weight

    def custom_factor(self, ticker, ftype):
        if ftype == 'marketcap':
            return self.get_value('metric',ticker,'marketcap')
        else:
            return self.get_value('fundamentals',ticker,ftype)



In [ ]:
input_list = list(engine.cache['fundamentals']['AAPL'].columns[5:])
unary_list = ['raw', 'rank', 'zscore']
binary_list = ['add', 'sub', 'mul', 'div']

results = []

config_set = []

idx = 0
while True:
    x1, x2 = random.sample(input_list,2)
    unary = random.sample(unary_list,1)[0]
    binary = random.sample(binary_list,1)[0]
    
    config = {
    'input1':x1,
    'input2':x2,
    'unary':unary,
    'binary':binary}
    
    if config in config_set:
        pass
    else:
        config_set.append(config)
    
    print('\n')
    mys = myStrategy(config)
    try:
        engine.run_backtest(mys, '2016-01-01','2021-08-31',period='Q')
        engine.stat(bench=None)
        print(engine.stat_dic)

        results.append([engine.stat_dic['sharpe'],engine.stat_dic['cagr'],config])
        idx += 1
    except:
        print('Passed!')




{'input1': 'shareswa', 'input2': 'roic', 'unary': 'zscore', 'binary': 'mul'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.453 / time elapsed:71.1 ===
[run_backtest] is executed in 71.57 seconds
{'cagr': 0.15821620525201383, 'vol': 0.20253419400855327, 'sharpe': 0.7811826838747643}


{'input1': 'ncf', 'input2': 'fxusd', 'unary': 'rank', 'binary': 'div'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.994 / time elapsed:70.3 ===
[run_backtest] is executed in 70.73 seconds
{'cagr': 0.12171883808950382, 'vol': 0.1936005038425456, 'sharpe': 0.6287113704440418}


{'input1': 'ncfbus', 'input2': 'investmentsnc', 'unary': 'raw', 'binary': 'sub'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.397 / time elapsed:67.4 ===
[run_backtest] is executed in 67.81 seconds
{'cagr': 0.1545532000983847, 'vol': 0.1837229779200489, 'sh

In [32]:
results.sort(key = lambda x:-x[0])

results

[[0.8827340485021868,
  0.18117440559120737,
  {'input1': 'netinc', 'input2': 'ebt', 'unary': 'zscore', 'binary': 'mul'}],
 [0.8230532196137099,
  0.15206874661484138,
  {'input1': 'equityavg',
   'input2': 'consolinc',
   'unary': 'rank',
   'binary': 'sub'}],
 [0.7525923232279663,
  0.12877337205149683,
  {'input1': 'de', 'input2': 'debtnc', 'unary': 'zscore', 'binary': 'add'}]]